<a href="https://colab.research.google.com/github/Achyuta-Harshavardhan/Spam-Classifier/blob/main/Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Necessary Libraries
import pandas as pd
import re
import nltk
import pickle
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# To read data from CSV file
messages = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/SMSSpamCollection.txt',sep='\t',names=["label","message"])

In [3]:
# Display first five records
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Cleaning the data
stemmer = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
# Bag of Words
cv = CountVectorizer(max_features = 2500)
x = cv.fit_transform(corpus).toarray()

In [6]:
# One Hot encoding
y=pd.get_dummies(messages['label'],drop_first=True)
y.head()

,spam
0,0
1,0
2,1
3,0
4,0


In [7]:
# Train and Test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [8]:
# Training model using Naive bayes classifier
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
# Prdictions
y_pred=spam_detect_model.predict(X_test)

In [10]:
# Accuracy Score
print(accuracy_score(y_pred,y_test))

0.9856502242152466


In [11]:
# Confusion Metrics
print(confusion_matrix(y_pred,y_test))

[[946   7]
 [  9 153]]


In [12]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       953
           1       0.96      0.94      0.95       162

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [15]:
file_name = "Spam_Classifier_MNB"
pickle.dump(spam_detect_model, open(file_name, 'wb'))